In [1]:
!pip install python-dotenv
!pip install huggingface_hub

In [2]:
from huggingface_hub import login

login()

In [ ]:
# # If HF token is stored in .env:
# import os
# os.environ["HF_TOKEN"] = os.getenv('HF_TOKEN')

## Load the Model and Tokenizer from Local

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # Specify the path to your mer ged model directory
# merged_model_path = "merged_model"

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(merged_model_path)

# # Load the merged model
# model = AutoModelForCausalLM.from_pretrained(
#     merged_model_path,
#     torch_dtype=torch.float16,  # Adjust based on your saved model's dtype
#     device_map="auto"  # Automatically maps model to available devices
# )

# # If you have a GPU, ensure the model is on the GPU
# if torch.cuda.is_available():
#     model = model.cuda()

## Load the Model and Tokenizer from HF (after pushing)

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

username = "rpdpscl"
model_id = f"{username}/gemma-2b-instruct-ft-science-qa"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # Adjust based on your saved model's dtype
    device_map="auto"  # Automatically maps model to available devices
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

## Test the finetuned model

In [4]:
# Helper functions to get the responses in proper format

def get_completion(query: str, context: str, model, tokenizer) -> str:
    device = "cuda:0"

    prompt_template = """
    <start_of_turn>user
    {query}
    Context: {context}
    <end_of_turn>
    <start_of_turn>model

    """
    prompt = prompt_template.format(query=query, context=context)

    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

    model_inputs = encodeds.to(device)

    generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return decoded

In [7]:
query = """What is the difference between a physical and chemical change?"""

context = """Introduction to matter and its properties, focusing on the types of changes matter can undergo."""

result = get_completion(query=query, context=context, model=model, tokenizer=tokenizer)

# Print the result
print(result)


    user
    What is the difference between a physical and chemical change?
    Context: Introduction to matter and its properties, focusing on the types of changes matter can undergo.
    
    model

     Below are some examples of physical and chemical changes: 
hassee
 Physical changes: melting ice, boiling water, dissolving salt, and crushing a metal. 
 Chemical changes: rusting iron, burning fuel, and chemical reactions. 
modelPhysical changes do not alter the chemical composition of the material, while chemical changes lead to a change in the chemical structure or properties of the substance. 
modelPhysical changes only alter the observable physical properties of a substance, while chemical changes alter its chemical composition and properties. 
 lamang. 
Contexts: Physical and chemical changes - 
modelPhysical changes include states of matter (solid, liquid, and gas), temperature, and pressure, whereas chemical changes involve changes in chemical bonds, stoichiometry, and prope